# dy/dx + 2x = 0

In [49]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math, random
import matplotlib.pyplot as plt
from scipy import special

# Define the number of outputs and the learning rate
n_input = 1
n_nodes_hl1 = 50
n_nodes_hl2 = 50
n_nodes_hl3 = 50
n_nodes_hl4 = 50
n_nodes_hl5 = 50
n_nodes_hl6 = 50
n_nodes_hl7 = 50
n_nodes_hl8 = 50
n_output = 1
learn_rate = 0.0002

# Boundary Conditions
A = 0

# training data
N = 9000
N1 = 500 # Number of datapoints to describe B.C's
a = 0
b = 1
diff = (b-a)/N
x = []
x1 = []
for i in range(N):
    x1.append(a+i*diff) 
    x.append(random.uniform(0, 1))
    if(i==N-1):
        for j in range(N1):
            x.append(1)
  
x1 = np.array(x1)
x1 = x1.reshape((N,1))
N = N + N1            
x = np.array(x)
x = x.reshape((N,1))

# Placeholders
x_ph = tf.placeholder('float', [None, 1],name='input')
y_ph = tf.placeholder('float', [None, 1],name='input')

# number of epochs
n_epochs = 500

# Define standard deviation for initialising weights and biases from normal distribution.
hl_sigma = 0.02

def neural_network_model(data):
    
    data = tf.cast(data, tf.float32)
    hidden_1_layer = {'weights': tf.Variable(tf.random.normal([n_input, n_nodes_hl1],stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl1], stddev=hl_sigma))}
    hidden_2_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl1, n_nodes_hl2], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl2], stddev=hl_sigma))}
    hidden_3_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl2, n_nodes_hl3], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl3], stddev=hl_sigma))}
    hidden_4_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl3, n_nodes_hl4], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl4], stddev=hl_sigma))}
    hidden_5_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl4, n_nodes_hl5], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl5], stddev=hl_sigma))}
    hidden_6_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl5, n_nodes_hl6], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl6], stddev=hl_sigma))}
    hidden_7_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl6, n_nodes_hl7], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl7], stddev=hl_sigma))}
    hidden_8_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl7, n_nodes_hl8], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl8], stddev=hl_sigma))}
    output_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl8, n_output], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_output], stddev=hl_sigma))}
           
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.tanh(l1)   
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.tanh(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.tanh(l3)
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.tanh(l4)
    l5 = tf.add(tf.matmul(l4, hidden_5_layer['weights']), hidden_5_layer['biases'])
    l5 = tf.nn.tanh(l5)
    l6 = tf.add(tf.matmul(l5, hidden_6_layer['weights']), hidden_6_layer['biases'])
    l6 = tf.nn.tanh(l6)
    l7 = tf.add(tf.matmul(l6, hidden_7_layer['weights']), hidden_7_layer['biases'])
    l7 = tf.nn.tanh(l7)
    l8 = tf.add(tf.matmul(l7, hidden_8_layer['weights']), hidden_8_layer['biases'])
    l8 = tf.nn.tanh(l8)
    output = tf.add(tf.matmul(l8, output_layer['weights']), output_layer['biases'], name='output')
    return tf.concat([tf.reshape(hidden_1_layer['weights'], [50]),tf.reshape(hidden_2_layer['weights'], [2500])], 0),output

def train_neural_network_batch():
    wts, u = neural_network_model(x_ph)  
    dudx = tf.gradients(u, x_ph)
    wts, v = neural_network_model(y_ph)
    
    cost = tf.reduce_mean(tf.square(dudx + 2*x_ph) + tf.square(tf.cast(tf.equal(x_ph,1), tf.float32)*(u-A)) )
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(n_epochs):
#             _, l = sess.run([optimizer,cost], feed_dict={x_ph:x})
            u1 = sess.run(u, feed_dict={x_ph:x})        
            v1 = sess.run(v, feed_dict={y_ph:u1})
            _, l = sess.run([optimizer,cost], feed_dict={x_ph:v1})
            print(len(v1))
#             print(l)
#             y = u.eval(session=sess)
            
#             if(epoch % 100 == 0):
#                 print('loss:-',l,', epoch:-',epoch)
#                 print(type(u))
#                 print(sess.run(wts))

        # Validation
        return sess.run(wts), sess.run(tf.squeeze(u),{x_ph:x1}), x1
    
                
wts, y_pred ,x_pred = train_neural_network_batch()
y_pred = y_pred.reshape(N-N1,1)
plt.plot(x_pred[0:N-N1], y_pred[0:N-N1], label ='NN')
plt.plot(x_pred[0:N-N1], -x_pred[0:N-N1]*x_pred[0:N-N1]+1, label ='Analytical')
plt.title("dT/dx + 2x = 0")
plt.legend()
plt.show()

9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500
9500


KeyboardInterrupt: 

In [13]:
x = tf.compat.v1.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.compat.v1.Session() as sess:
#     print(sess.run(y))  # ERROR: will fail because x was not fed.

    rand_array = np.random.rand(1024, 1024)
    print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.

[[253.72934 261.653   254.63487 ... 254.88412 262.72888 261.74814]
 [254.19106 262.34357 256.26352 ... 256.995   266.9292  269.97113]
 [241.87312 248.87424 245.28119 ... 252.10767 250.17346 256.80096]
 ...
 [252.92992 258.8196  252.55267 ... 262.6327  259.4683  265.95425]
 [254.6813  257.7068  251.2738  ... 255.58766 259.11414 262.31842]
 [247.67007 253.18756 246.97809 ... 259.94043 259.9941  258.73502]]


In [ ]:
# tensor to array
a = tf.constant([[1, 2], [3, 4]])
a.eval(session=tf.compat.v1.Session())

In [35]:
# tensor to array
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

a = tf.compat.v1.placeholder(tf.float32, shape=(2, 2))
b = tf.matmul(a, a)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = np.random.rand(2, 2)
    print(x)
    c = sess.run(b, feed_dict={a:x})
    print(c)
    print(type(c))


[[0.02144775 0.72002786]
 [0.39256777 0.39280851]]
[[0.2831197  0.29827604]
 [0.16262364 0.43695822]]
<class 'numpy.ndarray'>


In [ ]:
# wt = wts.reshape(50,1)
wts

# d2y/dx2 = 0

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math, random
import matplotlib.pyplot as plt
from scipy import special

# Define the number of outputs and the learning rate
n_input = 1
n_nodes_hl1 = 50
n_nodes_hl2 = 50
n_nodes_hl3 = 50
n_nodes_hl4 = 50
n_nodes_hl5 = 50
n_nodes_hl6 = 50
n_nodes_hl7 = 50
n_nodes_hl8 = 50
n_output = 1
learn_rate = 0.0002

# Boundary Conditions
A = 1
B = 0

# training data
N = 9000
N1 = 500 # Number of datapoints to describe B.C's
a = 0
b = 1
diff = (b-a)/N
x = []
x1 = []
for i in range(N):
    x1.append(a+i*diff) 
    x.append(random.uniform(0, 1))
    if(i==N-1):
        for j in range(N1):
            x.append(0)
            x.append(1)
            
x1 = np.array(x1)
x1 = x1.reshape((N,1))
N = N + 2*N1            
x = np.array(x)
x = x.reshape((N,1))

# Placeholders
x_ph = tf.placeholder('float', [None, 1],name='input')
y_ph = tf.placeholder('float')

# number of epochs
n_epochs = 5000

# Define standard deviation for initialising weights and biases from normal distribution.
hl_sigma = 0.02

def neural_network_model(data):
    
    data = tf.cast(data, tf.float32)
    hidden_1_layer = {'weights': tf.Variable(tf.random.normal([n_input, n_nodes_hl1],stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl1], stddev=hl_sigma))}
    hidden_2_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl1, n_nodes_hl2], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl2], stddev=hl_sigma))}
    hidden_3_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl2, n_nodes_hl3], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl3], stddev=hl_sigma))}
    hidden_4_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl3, n_nodes_hl4], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl4], stddev=hl_sigma))}
    hidden_5_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl4, n_nodes_hl5], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl5], stddev=hl_sigma))}
    hidden_6_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl5, n_nodes_hl6], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl6], stddev=hl_sigma))}
    hidden_7_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl6, n_nodes_hl7], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl7], stddev=hl_sigma))}
    hidden_8_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl7, n_nodes_hl8], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl8], stddev=hl_sigma))}
    output_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl8, n_output], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_output], stddev=hl_sigma))}
           
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.tanh(l1)   
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.tanh(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.tanh(l3)
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.tanh(l4)
    l5 = tf.add(tf.matmul(l4, hidden_5_layer['weights']), hidden_5_layer['biases'])
    l5 = tf.nn.tanh(l5)
    l6 = tf.add(tf.matmul(l5, hidden_6_layer['weights']), hidden_6_layer['biases'])
    l6 = tf.nn.tanh(l6)
    l7 = tf.add(tf.matmul(l6, hidden_7_layer['weights']), hidden_7_layer['biases'])
    l7 = tf.nn.tanh(l7)
    l8 = tf.add(tf.matmul(l7, hidden_8_layer['weights']), hidden_8_layer['biases'])
    l8 = tf.nn.tanh(l8)
    output = tf.add(tf.matmul(l8, output_layer['weights']), output_layer['biases'], name='output')
    return hidden_1_layer['weights'], output

def train_neural_network_batch():
    wts, u = neural_network_model(x_ph)  
    dudx = tf.gradients(u, x_ph)
    dudx2 = tf.gradients(tf.gradients(u, x_ph), x_ph)
    
    
    cost = tf.reduce_mean(tf.square(dudx2) +  tf.square(tf.cast(tf.equal(x_ph,0), tf.float32)*(u-A)) + tf.square(tf.cast(tf.equal(x_ph,1), tf.float32)*(u-B)) )
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(n_epochs):
            _, l = sess.run([optimizer,cost], feed_dict={x_ph:x})
            
            if(epoch % 100 == 0):
                print('loss:-',l,', epoch:-',epoch)
                if(l<=0.00005):
                    break

        # Validation
        return wts, sess.run(tf.squeeze(u),{x_ph:x1}), x1
    
                
wts, y_pred ,x_pred = train_neural_network_batch()
y_pred = y_pred.reshape(N-2*N1,1)
plt.plot(x_pred[0:N-2*N1], y_pred[0:N-2*N1], label ='NN')
plt.plot(x_pred[0:N-2*N1], x_pred[0:N-2*N1]*(B-A) + A, label ='Analytical')
plt.legend()
plt.show()

# k*d2T/dx2 + s = 0

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math, random
import matplotlib.pyplot as plt
from scipy import special

# Define the number of outputs and the learning rate
n_input = 1
n_nodes_hl1 = 50
n_nodes_hl2 = 50
n_nodes_hl3 = 50
n_nodes_hl4 = 50
n_nodes_hl5 = 50
n_nodes_hl6 = 50
n_nodes_hl7 = 50
n_nodes_hl8 = 50
n_output = 1
learn_rate = 0.0006

# Boundary Conditions
A = 0
B = 1
C = 2.0

# training data
N = 9000
N1 = 900 # Number of datapoints to describe B.C's
a = 0
b = 1
diff = (b-a)/N
x = []
for i in range(N):
    x.append(a+i*diff)  
    if(i==N-1):
        for j in range(N1):
            x.append(0)
            x.append(1)
            
N = N + 2*N1            
x = np.array(x)
x = x.reshape((N,1))

# Placeholders
x_ph = tf.placeholder('float', [None, 1],name='input')
y_ph = tf.placeholder('float')

# number of epochs
n_epochs = 5000

# Define standard deviation for initialising weights and biases from normal distribution.
hl_sigma = 0.02
me = 0.0
def neural_network_model(data):
    
    data = tf.cast(data, tf.float32)
    hidden_1_layer = {'weights': tf.Variable(tf.random.normal([n_input, n_nodes_hl1], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl1], mean = me, stddev=hl_sigma))}
    hidden_2_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl1, n_nodes_hl2], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl2], mean = me, stddev=hl_sigma))}
    hidden_3_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl2, n_nodes_hl3], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl3], mean = me, stddev=hl_sigma))}
    hidden_4_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl3, n_nodes_hl4], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl4], mean = me, stddev=hl_sigma))}
    hidden_5_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl4, n_nodes_hl5], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl5], mean = me, stddev=hl_sigma))}
    hidden_6_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl5, n_nodes_hl6], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl6], mean = me, stddev=hl_sigma))}
    hidden_7_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl6, n_nodes_hl7], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl7], mean = me, stddev=hl_sigma))}
    hidden_8_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl7, n_nodes_hl8], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl8], mean = me, stddev=hl_sigma))}
    output_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl8, n_output], mean = me, stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_output], mean = me, stddev=hl_sigma))}
           
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.tanh(l1)   
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.tanh(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.tanh(l3)
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.tanh(l4)
    l5 = tf.add(tf.matmul(l4, hidden_5_layer['weights']), hidden_5_layer['biases'])
    l5 = tf.nn.tanh(l5)
    l6 = tf.add(tf.matmul(l5, hidden_6_layer['weights']), hidden_6_layer['biases'])
    l6 = tf.nn.tanh(l6)
    l7 = tf.add(tf.matmul(l6, hidden_7_layer['weights']), hidden_7_layer['biases'])
    l7 = tf.nn.tanh(l7)
    l8 = tf.add(tf.matmul(l7, hidden_8_layer['weights']), hidden_8_layer['biases'])
    l8 = tf.nn.tanh(l8)
    output = tf.add(tf.matmul(l8, output_layer['weights']), output_layer['biases'], name='output')
    return output

def train_neural_network_batch():
    u = neural_network_model(x_ph)  
    dudx = tf.gradients(u, x_ph)
    dudx2 = tf.gradients(tf.gradients(u, x_ph), x_ph)
    
    
    cost = tf.reduce_mean(tf.square(dudx2 + tf.constant(C)) +  tf.square(tf.cast(tf.equal(x_ph,0), tf.float32)*(u-A)) + tf.square(tf.cast(tf.equal(x_ph,1), tf.float32)*(u-B)) )
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(n_epochs):
            _, l = sess.run([optimizer,cost], feed_dict={x_ph:x})
            
            if(epoch % 100 == 0):
                print('loss:-',l,', epoch:-',epoch)
                if(l<=0.0006):
                    break

        # Validation
        return sess.run(tf.squeeze(u),{x_ph:x}), x
    
                
y_pred ,x_pred = train_neural_network_batch()
y_pred = y_pred.reshape(N,1)
plt.plot(x_pred[0:N-2*N1], y_pred[0:N-2*N1], label ='NN')
plt.plot(x_pred[0:N-2*N1], -C*x_pred[0:N-2*N1]*x_pred[0:N-2*N1]/2 + (B - A + C/2)*x_pred[0:N-2*N1] + A, label ='Analytical')
plt.legend()
plt.show()

# dT/dt - (alpha)*(d2T/dX2) = 0

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math, random
import matplotlib.pyplot as plt
from scipy import special

# Define the number of outputs and the learning rate
n_input = 2
n_nodes_hl1 = 30
n_nodes_hl2 = 30
n_nodes_hl3 = 30
n_nodes_hl4 = 30
n_nodes_hl5 = 30
n_nodes_hl6 = 30
n_nodes_hl7 = 30
n_nodes_hl8 = 30
n_output = 1
learn_rate = 0.0002

# Boundary Conditions
A = 0
B = 1
C = 0.1
T = 1

# training data
N = 9000
N1 = 100 # Number of datapoints to describe B.C's
a = 0
b = 1
diff = (b-a)/N
x = []
t = []
x1 = []
t1 = []
for i in range(N):
    x1.append(a+i*diff) 
    t1.append(T)
    x.append(random.uniform(0, 1))
    t.append(random.uniform(0, 1))
    if(i==N-1):
        for j in range(N1):
            x.append(0)
            t.append(random.uniform(0, 1))
            x.append(1)
            t.append(random.uniform(0, 1))
            x.append(random.uniform(0, 1))
            t.append(0)
            
x1 = np.array(x1)
x1 = x1.reshape((N,1))
t1 = np.array(t1)
t1 = t1.reshape((N,1))
N = N + 3*N1            
x = np.array(x)
x = x.reshape((N,1))
t = np.array(t)
t = t.reshape((N,1))

# Placeholders
x_ph = tf.placeholder('float', [None, 1],name='input')
t_ph = tf.placeholder('float', [None, 1],name='input')
y_ph = tf.placeholder('float')

# number of epochs
n_epochs = 10000

# Define standard deviation for initialising weights and biases from normal distribution.
hl_sigma = 0.1

def neural_network_model(x2, t2):
    data = tf.concat([x2, t2], 1)
    data = tf.cast(data, tf.float32)
    hidden_1_layer = {'weights': tf.Variable(tf.random.normal([n_input, n_nodes_hl1],stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl1], stddev=hl_sigma))}
    hidden_2_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl1, n_nodes_hl2], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl2], stddev=hl_sigma))}
    hidden_3_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl2, n_nodes_hl3], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl3], stddev=hl_sigma))}
    hidden_4_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl3, n_nodes_hl4], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl4], stddev=hl_sigma))}
    hidden_5_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl4, n_nodes_hl5], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl5], stddev=hl_sigma))}
    hidden_6_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl5, n_nodes_hl6], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl6], stddev=hl_sigma))}
    hidden_7_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl6, n_nodes_hl7], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl7], stddev=hl_sigma))}
    hidden_8_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl7, n_nodes_hl8], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl8], stddev=hl_sigma))}
    output_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl8, n_output], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_output], stddev=hl_sigma))}
           
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.tanh(l1)   
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.tanh(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.tanh(l3)
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.tanh(l4)
    l5 = tf.add(tf.matmul(l4, hidden_5_layer['weights']), hidden_5_layer['biases'])
    l5 = tf.nn.tanh(l5)
    l6 = tf.add(tf.matmul(l5, hidden_6_layer['weights']), hidden_6_layer['biases'])
    l6 = tf.nn.tanh(l6)
    l7 = tf.add(tf.matmul(l6, hidden_7_layer['weights']), hidden_7_layer['biases'])
    l7 = tf.nn.tanh(l7)
    l8 = tf.add(tf.matmul(l7, hidden_8_layer['weights']), hidden_8_layer['biases'])
    l8 = tf.nn.tanh(l8)
    output = tf.add(tf.matmul(l8, output_layer['weights']), output_layer['biases'], name='output')
    return output

def train_neural_network_batch():
    u = neural_network_model(x_ph,t_ph)  
    dudx = tf.gradients(u, x_ph)
    dudx2 = tf.gradients(tf.gradients(u, x_ph), x_ph)
    dudt = tf.gradients(u, t_ph)
    cost = tf.reduce_mean(tf.square(dudt-tf.constant(C)*dudx2) + tf.square(tf.cast(tf.equal(x_ph,0), tf.float32)*(u-A)) + tf.square(tf.cast(tf.equal(x_ph,1), tf.float32)*(u-B)) + tf.square(tf.cast(tf.equal(t_ph,0), tf.float32)*(u-A)))
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(n_epochs):
            _, l = sess.run([optimizer,cost], feed_dict={x_ph:x, t_ph:t})
            
            if(epoch % 100 == 0):
                print('loss:-',l,', epoch:-',epoch)
                if(l<=0.00005):
                    break

        # Validation
        return sess.run(tf.squeeze(u),{x_ph:x1, t_ph:t1}), x1
    
# Run the code                              
y_pred ,x_pred = train_neural_network_batch()
y_pred = y_pred.reshape(N-3*N1,1)

# Analytical
y1 = []
T = 0.4
i = 1
y1 = -2*(-1)**(i+1)/(i*np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)
for i in range(2, 49, 1):
    y1 += -2*(-1)**(i+1)/(i*np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)   
y1 += x1

# Plot
plt.plot(x_pred[0:N-3*N1], y_pred[0:N-3*N1], label ='NN')
plt.plot(x1, y1, label ='Analytical')
plt.title("dT/dt - (c)*(d2T/dx2) = 0, c = 0.1, t = 1")
plt.legend()
plt.show()

In [ ]:
import pandas as  pd
final = pd.DataFrame(list(zip(x_pred[0:N-3*N1], y_pred[0:N-3*N1])),columns =['Length','Temperature'])
final
final.to_excel("1-D_Unsteady_Conduction_t=1_A=0_B=1_ini=0.xlsx")

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math, random
import matplotlib.pyplot as plt
from scipy import special
#import tensorflow_probability as tfp

# Define the number of outputs and the learning rate
n_input = 2
n_nodes_hl1 = 30
n_nodes_hl2 = 30
n_nodes_hl3 = 30
n_nodes_hl4 = 30
n_nodes_hl5 = 30
n_nodes_hl6 = 30
n_nodes_hl7 = 30
n_nodes_hl8 = 30
n_output = 1
learn_rate = 0.0002

# Boundary Conditions
A = 0
B = 1
C = 0.15
T = 0.2

# training data
N = 9000
N1 = 100 # Number of datapoints to describe B.C's
a = 0
b = 1
diff = (b-a)/N
x = []
t = []
x1 = []
t1 = []
for i in range(N):
    x1.append(a+i*diff) 
    t1.append(T)
    x.append(random.uniform(0, 1))
    t.append(random.uniform(0, 1))
    if(i==N-1):
        for j in range(N1):
            x.append(0)
            t.append(random.uniform(0, 1))
            x.append(1)
            t.append(random.uniform(0, 1))
            x.append(random.uniform(0, 1))
            t.append(0)
            
x1 = np.array(x1)
x1 = x1.reshape((N,1))
t1 = np.array(t1)
t1 = t1.reshape((N,1))
N = N + 3*N1            
x = np.array(x)
x = x.reshape((N,1))
t = np.array(t)
t = t.reshape((N,1))

# Placeholders
x_ph = tf.placeholder('float', [None, 1],name='input')
t_ph = tf.placeholder('float', [None, 1],name='input')
y_ph = tf.placeholder('float')

# number of epochs
n_epochs = 7000

# Define standard deviation for initialising weights and biases from normal distribution.
hl_sigma = 0.1

def neural_network_model(x2, t2):
    data = tf.concat([x2, t2], 1)
    data = tf.cast(data, tf.float32)
    hidden_1_layer = {'weights': tf.Variable(tf.random.normal([n_input, n_nodes_hl1],stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl1], stddev=hl_sigma))}
    hidden_2_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl1, n_nodes_hl2], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl2], stddev=hl_sigma))}
    hidden_3_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl2, n_nodes_hl3], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl3], stddev=hl_sigma))}
    hidden_4_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl3, n_nodes_hl4], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl4], stddev=hl_sigma))}
    hidden_5_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl4, n_nodes_hl5], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl5], stddev=hl_sigma))}
    hidden_6_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl5, n_nodes_hl6], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl6], stddev=hl_sigma))}
    hidden_7_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl6, n_nodes_hl7], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl7], stddev=hl_sigma))}
    hidden_8_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl7, n_nodes_hl8], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl8], stddev=hl_sigma))}
    output_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl8, n_output], stddev=hl_sigma)),
                      'biases': tf.Variable(tf.random.normal([n_output], stddev=hl_sigma))}
           
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.tanh(l1)   
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.tanh(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.tanh(l3)
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']), hidden_4_layer['biases'])
    l4 = tf.nn.tanh(l4)
    l5 = tf.add(tf.matmul(l4, hidden_5_layer['weights']), hidden_5_layer['biases'])
    l5 = tf.nn.tanh(l5)
    l6 = tf.add(tf.matmul(l5, hidden_6_layer['weights']), hidden_6_layer['biases'])
    l6 = tf.nn.tanh(l6)
    l7 = tf.add(tf.matmul(l6, hidden_7_layer['weights']), hidden_7_layer['biases'])
    l7 = tf.nn.tanh(l7)
    l8 = tf.add(tf.matmul(l7, hidden_8_layer['weights']), hidden_8_layer['biases'])
    l8 = tf.nn.tanh(l8)
    output = tf.add(tf.matmul(l8, output_layer['weights']), output_layer['biases'], name='output')
    return output

def train_neural_network_batch():
    u = neural_network_model(x_ph,t_ph)  
    dudx = tf.gradients(u, x_ph)
    dudx2 = tf.gradients(tf.gradients(u, x_ph), x_ph)
    dudt = tf.gradients(u, t_ph)
    cost = tf.reduce_mean(tf.square(dudt-tf.constant(C)*dudx2) + tf.square(tf.cast(tf.equal(x_ph,0), tf.float32)*(u-A)) + tf.square(tf.cast(tf.equal(x_ph,1), tf.float32)*(u-A)) + tf.square(tf.cast(tf.equal(t_ph,0), tf.float32)*(u-B)))
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(n_epochs):
            _, l = sess.run([optimizer,cost], feed_dict={x_ph:x, t_ph:t})
            
            if(epoch % 100 == 0):
                print('loss:-',l,', epoch:-',epoch)
                if(l<=0.0005):
                    break

        # Validation
        return sess.run(tf.squeeze(u),{x_ph:x1, t_ph:t1}), x1
    
                          
y_pred ,x_pred = train_neural_network_batch()
y_pred = y_pred.reshape(N-3*N1,1)

# Analytical Solution
y1 = []
i = 1
y1 = (4/np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)
for i in range(3, 21, 2):
    y1 += (4/np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)

# Plot 
plt.plot(x_pred[0:N-3*N1], y_pred[0:N-3*N1], label ='NN')
plt.plot(x1, y1, label ='Analytical')
s1 = "dT/dt - (c)*(d2T/dx2) = 0, c = "
s2 = str(C)
s3 = ", t = "
s4 = str(T)
plt.title(s1+s2+s3+s4)
plt.legend()
plt.show()

In [ ]:
i = (N-3*N1)/2
y_pred[4500]

In [ ]:
i

In [ ]:
y1 = []
T = 1
i = 1
y1 = -2*(-1)**(i+1)/(i*np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)
for i in range(2, 49, 1):
    y1 += -2*(-1)**(i+1)/(i*np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)
    
y1 += x1

In [ ]:
plt.plot(x_pred[0:N-3*N1], y_pred[0:N-3*N1], label ='NN')
plt.plot(x1, y1, label ='Analytical')
s1 = "dT/dt - (c)*(d2T/dx2) = 0, c = "
s2 = str(C)
s3 = ", t = "
s4 = str(T)
plt.title(s1+s2+s3+s4)
plt.legend()
plt.show()

In [ ]:
y1[0]

In [ ]:
i = 1
y1 = (4/np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)
y1

In [ ]:
i = 2
y1 += (4/np.pi)*np.sin(i*np.pi*x1)*np.exp(-i**2*np.pi**2*C*T)

In [ ]:
y1